In [1]:
from bvh import Bvh
with open('D:\\tuc\\exam10\\Thesis\\Dataset\\07_06.bvh') as f:
    mocap = Bvh(f.read())



In [4]:
mocap.get_joints_names()

['reference',
 'Hips',
 'LHipJoint',
 'LeftUpLeg',
 'LeftLeg',
 'LeftFoot',
 'LeftToeBase',
 'RHipJoint',
 'RightUpLeg',
 'RightLeg',
 'RightFoot',
 'RightToeBase',
 'LowerBack',
 'Spine',
 'Spine1',
 'Neck',
 'Neck1',
 'Head',
 'LeftShoulder',
 'LeftArm',
 'LeftForeArm',
 'LeftHand',
 'LeftFingerBase',
 'LeftHandIndex1',
 'LThumb',
 'RightShoulder',
 'RightArm',
 'RightForeArm',
 'RightHand',
 'RightFingerBase',
 'RightHandIndex1',
 'RThumb']

In [2]:
# reading the data from the file
from typing import Counter


data = open('D:\\tuc\\exam10\\Thesis\\Dataset\\07_06.bvh', 'r')
Lines = data.readlines()
Edited = []
for n,line in enumerate(Lines) :
    if n == 1 or n == 2 or n == 3 or n == 4 or n == 188:
        continue
    if n>191 :
      print(line)
      break
      continue
    else:
       # print(line)
        Edited.append(str(line))

0 0 0 0 0 -0 42.6231 91.7159 -227.652 0 0 -0 0 0 0 0 0 -0 10.4754 -9.81838 4.79639 -21 0 -0 13.368 -36.7282 0 0 0 -0 14.2955 -39.2765 0 0 0 -0 0.899435 -2.47117 10.9787 0 0 -0 0 0 0 0 0 -0 -9.49936 -9.81838 4.79639 21 0 -0 -13.8124 -37.9492 0 0 0 -0 -13.7635 -37.8148 0 0 0 -0 -1.17708 -3.23396 11.4114 0 0 -0 0 0 0 0 0 -0 -0.0880526 12.6418 -0.20952 0 0 -0 0.309878 12.3739 -1.07729 0 0 -0 0 0 0 -0 -16 -0 -0.700075 8.08097 1.55701 0 21 -0 1.00781 8.25059 -1.83883 0 11 -0 0 0 0 0 0 -0 18.4939 4.83353 0.304572 -8 0 -0 28.0388 2.00529e-14 0 0 0 -0 18.9511 -2.00529e-14 0 0 0 -0 0 0 0 0 0 -0 4.49842 0 0 0 0 -0 0 0 0 0 0 -0 0 0 0 0 0 -0 -18.3124 5.75921 1.95793 8 0 -0 -29.4558 -2.00529e-14 0 0 0 -0 -18.9936 2.00529e-14 0 0 0 -0 0 0 0 0 0 -0 -3.50201 -2.00529e-14 0 0 0 -0 0 0 0 0 0 -0 

